# Demonstrating `mobile-env:smart-city`

`mobile-env` is a simple and open environment for training, testing, and evaluating a decentralized metaverse environment.

* `mobile-env:smart-city` is written in pure Python
* It allows simulating various scenarios with moving users in a cellular network with a single base station and multiple stationary sensors
* `mobile-env:smart-city` implements the standard [Gymnasium](https://gymnasium.farama.org/) (previously [OpenAI Gym](https://gym.openai.com/)) interface such that it can be used with all common frameworks for reinforcement learning
* `mobile-env:smart-city` is not restricted to reinforcement learning approaches but can also be used with conventional control approaches or dummy benchmark algorithms
* It can be configured easily (e.g., adjusting number and movement of users, properties of cells, etc.)
* It is also easy to extend `mobile-env:smart-city`, e.g., implementing different observations, actions, or reward

As such `mobile-env:smart-city` is a simple platform to ...


**Demonstration Steps:**

This demonstration consists of the following steps:

1. Installation and usage of `mobile-env` with dummy actions
2. Configuration of `mobile-env` and adjustment of the observation space (optional)
3. Training a single-agent reinforcement learning approach with [`stable-baselines3`](https://github.com/DLR-RM/stable-baselines3)

In [ ]:
# First, install stable baselines; only SB3 v2.0.0+ supports Gymnasium
%pip install stable-baselines3==2.0.0 tensorboard

In [ ]:
# Importing necessary libraries
import gymnasium
import mobile_env
import numpy as np
from stable_baselines3 import PPO
from stable_baselines3.ppo import MlpPolicy
from stable_baselines3.common.env_checker import check_env

# predefined small scenarios
from mobile_env.scenarios.smart_city import MComSmartCity

# easy access to the default configuration
MComSmartCity.default_config()

In [3]:
from gymnasium.envs.registration import register

# Register the new environment
register(
    id='mobile-smart_city-smart_city_handler-rl-v0',
    entry_point='mobile_env.scenarios.smart_city:MComSmartCity',  # Adjust this if the entry point is different
    kwargs={'config': {}, 'render_mode': None}
)

In [ ]:
import gymnasium as gym

# List all registered environments
env_specs = gym.envs.registry.keys()
print(env_specs)

# Verify your specific environment is listed
assert 'mobile-smart_city-smart_city_handler-rl-v0' in env_specs, "Environment not registered correctly"
print("Environment 'mobile-smart_city-smart_city_handler-rl-v0' registered successfully!")

In [ ]:
# create a small mobile environment for a single, centralized control agent
# pass rgb_array as render mode so the env can be rendered inside the notebook
env = gymnasium.make("mobile-smart_city-smart_city_handler-rl-v0", render_mode="rgb_array")

print(f"\nSmart city environment for RL with {env.NUM_USERS} users, {env.NUM_SENSORS} sensors and {env.NUM_STATIONS} cells.")

In [ ]:
# Step 4: Train a Single-Agent Reinforcement Learning

import os
from stable_baselines3 import PPO
from stable_baselines3.common.callbacks import EvalCallback, CheckpointCallback
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.env_util import make_vec_env

# Define the model for training (PPO algorithm)
model = PPO(MlpPolicy, env, tensorboard_log='results_sb', verbose=1)

# Callback: Save a checkpoint of the model at regular intervals
checkpoint_callback = CheckpointCallback(save_freq=10000, save_path='./checkpoints/',
                                         name_prefix='ppo_model_checkpoint')

# Callback: Evaluate the model every 5000 timesteps and log the results
eval_callback = EvalCallback(env, best_model_save_path='./logs/best_model',
                             log_path='./logs/', eval_freq=500,
                             deterministic=True, render=False)

# Train the model for a set number of timesteps (modify to suit your needs)
model.learn(total_timesteps=10000)

# Save the trained model for future use
model.save("ppo_mobile_env")

In [ ]:
# Step 5: Test the Trained Model

# Load the saved model
model = PPO.load("ppo_mobile_env")

In [ ]:
# Step 6: Test the model in the environment

obs = env.reset()
for step in range(200):
    # Use the trained model to predict the action
    action, _states = model.predict(obs)
    
    # Take the action in the environment
    obs, reward, done, info = env.step(action)
    
    # Print observation and reward
    print(f"Step {step+1} | Action: {action} | Observation: {obs} | Reward: {reward}")
    
    # Break the loop if the episode is finished
    if done:
        print("Episode finished")
        break

In [ ]:
# Step 7: Plot Results (Optional)

# Example of plotting some metrics (assuming the environment logs metrics like rewards, delays, etc.)
import matplotlib.pyplot as plt

# Example plotting of dummy reward over episodes (assuming we have a list of rewards)
# This is just an illustrative example - you'll need to replace this with your own logic for recording rewards
rewards = [np.random.uniform(-1, 1) for _ in range(100)]  # Replace with actual data

plt.plot(rewards)
plt.title("Reward Over Episodes")
plt.xlabel("Episode")
plt.ylabel("Reward")
plt.show()